# 0. Imports

In [1]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os

# 1. Data Sets

In [2]:
songs = pd.read_csv('../../data/songs_short.csv')
user = pd.read_csv('../../data/user.csv')
history = pd.read_csv('../../data/listening history.csv')

In [3]:
print('Songs:')
print(songs.head())

print('Users:')
print(user.head())

print('User listening history:')
print(history.head())

Songs:
               spotify_id                          Song         Artist  Rank  \
0  4H2TRR9FjnnIwxGnIt9stO                  Killin' Time    Clint Black     1   
1  6iy4PoAuZBMvtrlDX4VxC7               Check Yes Or No  George Strait     2   
2  3oW6SWwGqiZSPTiAp7ZQoH  It's A Great Day To Be Alive   Travis Tritt     3   
3  7lUE02KHkZM44BZgjCaWRO            Meet in the Middle    Diamond Rio     4   
4  1ecIqo8pOc8S6y3l80iGxd                         Fancy  Reba McEntire     5   

     Genre  
0  country  
1  country  
2  country  
3  country  
4  country  
Users:
     UserID  Password                 Name Genre_1  Genre_2  Genre_3
0     sibyl  7S02NXS3           Sibyl Bode    rock      r&b      pop
1   eleazar  7S02NXS4    Eleazar Hettinger     rap     soul  hip hop
2  geovanny  7S02NXS5        Geovanny Lang     rap  country     soul
3     micah  7S02NXS6    Micah Satterfield    jazz      r&b     soul
4  courtney  7S02NXS7  Courtney Schowalter    soul     folk    metal
User listenin

# 2. h2o Model

In [4]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-985GNVY6C2hOrmixYxL9uJ9DtpLZFrNb42NmPAf5TN4WXXgu',
)

In [5]:
# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="Music Reccommender",
)

In [6]:
# Convert Data to Readable Format
songs_txt = songs.to_csv(index=False)
user_txt = user.to_csv(index=False)
history_txt = history.to_csv(index=False)

In [7]:
# Upload
songs_data = client.upload('songs_short.txt', songs_txt.encode())
user_data = client.upload('user.txt', user_txt.encode())
history_data = client.upload('listening_history.txt', history_txt.encode())

# Ingest the uploaded data
client.ingest_uploads(collection_id, [songs_data, user_data, history_data])

Job(id='908aff7d-eb7c-4732-8cf2-99d990d1bd9f', name='Adding documents', passed=0.8571428571428571, failed=0.14285714285714285, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 31, 12, 54, 57, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='11beed69784441a39f2df02f0c3397c7', status='Indexing done.'), JobStatus(id='9e21acfab9f44dd386a0106f2766b55b', status='Indexing done.')], errors=['Job crashed: [Errno 503] Not Ready or Lagging'], last_update_date=datetime.datetime(2024, 3, 31, 12, 55, 23, tzinfo=TzInfo(UTC)), duration='26s', duration_seconds=26.0)

# 3. ChatBot Page

In [7]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [9]:
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="I am user sibyl. Reccomend me songs similar to my taste",
        system_prompt = 'Assume music and song to be the same word. When a question as for a similar music, recommend less than 5 music unless told otherwise. Find similar music based on genre and other factors such as danceability, loudness, speechiness and more. Just return the song name, unless stated otherwise.' ,
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on your user information, you enjoy rock, r&b, and pop music. Here are some song recommendations for you:

1. "Bohemian Rhapsody" by Queen (rock)
2. "Purple Rain" by Prince (r&b/pop)
3. "Mr. Brightside" by The Killers (rock/pop)
4. "I Want it That Way" by Backstreet Boys (pop)
5. "No Scrubs" by TLC (r&b)
